<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map

# Create a base map centered around the USA with an appropriate zoom level
spacex_map = folium.Map(location=[29.559684, -95.083097], zoom_start=5)

# Coordinates for each launch site
launch_sites = [
    {"name": "CCAFS SLC 40", "lat": 28.561857, "lon": -80.577366},
    {"name": "VAFB SLC 4E", "lat": 34.632093, "lon": -120.610829},
    {"name": "KSC LC 39A", "lat": 28.573255, "lon": -80.646895},
    {"name": "CCAFS LC 40", "lat": 28.562302, "lon": -80.577356},
]

# Add markers for each launch site
for site in launch_sites:
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=f"Launch Site: {site['name']}",
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(spacex_map)

# Display the map
spacex_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
import folium
from folium import DivIcon

# Coordinates for the map's initial center (NASA Kennedy Space Center)
nasa_coordinate = [28.573255, -80.646895]

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Launch site data (with names, latitudes, and longitudes)
launch_sites = [
    {"name": "CCAFS SLC 40", "lat": 28.561857, "lon": -80.577366},
    {"name": "VAFB SLC 4E", "lat": 34.632093, "lon": -120.610829},
    {"name": "KSC LC 39A", "lat": 28.573255, "lon": -80.646895},
    {"name": "CCAFS LC 40", "lat": 28.562302, "lon": -80.577356},
]

# Add circles and markers for each launch site
for site in launch_sites:
    # Add a circle for each launch site
    folium.Circle(
        location=[site["lat"], site["lon"]],
        radius=1000,  # Radius in meters
        color="#000000",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.4,
    ).add_child(folium.Popup(f"{site['name']}")).add_to(site_map)

    # Add a marker with a label for each launch site
    folium.map.Marker(
        location=[site["lat"], site["lon"]],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["name"]}</b></div>',
        ),
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
# Example dataframe containing launch information
# Replace this with your actual SpaceX launch data
import pandas as pd

launch_data = pd.DataFrame({
    "LaunchSite": ["CCAFS SLC 40", "VAFB SLC 4E", "KSC LC 39A", "CCAFS SLC 40", "VAFB SLC 4E"],
    "Latitude": [28.561857, 34.632093, 28.573255, 28.562302, 34.632093],
    "Longitude": [-80.577366, -120.610829, -80.646895, -80.577356, -120.610829],
    "Class": [1, 0, 1, 1, 0]  # 1 = Success, 0 = Failure
})

# Add success/failure markers to the map
for _, launch in launch_data.iterrows():
    # Marker color based on success (green) or failure (red)
    marker_color = "green" if launch["Class"] == 1 else "red"

    # Add marker
    folium.Marker(
        location=[launch["Latitude"], launch["Longitude"]],
        icon=folium.Icon(color=marker_color),
        popup=f"Site: {launch['LaunchSite']}<br>Outcome: {'Success' if launch['Class'] == 1 else 'Failure'}",
    ).add_to(site_map)

# Display the map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
#marker_cluster = MarkerCluster()
import folium
from folium.plugins import MarkerCluster

# Initialize a map centered at NASA coordinates
nasa_coordinate = [29.559684, -95.083097]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Extract launch site coordinates
    lat = record['Lat']
    lon = record['Long']
    outcome = record['class']
    
    # Define marker color based on launch success or failure
    marker_color = 'green' if outcome == 1 else 'red'
    
    # Add a marker to the cluster
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=marker_color),
        popup=f"Launch Outcome: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Define a function to assign marker colors based on the class value
def assign_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

# Apply the function to the class column and create a new column marker_color
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the first few rows to verify
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to the current site_map
site_map.add_child(marker_cluster)

# Iterate over each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Extract latitude, longitude, and marker color
    lat = record['Lat']
    lon = record['Long']
    color = record['marker_color']

    # Create a Marker object with its coordinate and properties
    marker = folium.Marker(
        location=[lat, lon],  # Set marker location
        icon=folium.Icon(color='white', icon_color=color),  # Set icon color
        popup=f"Launch Site: {record['Launch Site']}, Success: {record['class']}"  # Popup info
    )

    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math
import pandas as pd

# Function to calculate distance using the Haversine formula (in kilometers)
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radius of the Earth in kilometers (mean radius)
    R = 6371.0
    distance = R * c
    return distance

# Example proximity point (this could be another site or any point you're interested in)
# e.g., a coast or a nearby city
proximity_lat = 28.5721  # Replace with actual latitude of the proximity point
proximity_lon = -80.648  # Replace with actual longitude of the proximity point

# Assuming spacex_df is your dataframe with 'Lat' and 'Long' columns for launch site coordinates
spacex_df['distance_to_proximity'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], proximity_lat, proximity_lon), axis=1
)

# Display the dataframe with the new column for distances
spacex_df[['Launch Site', 'Lat', 'Long', 'distance_to_proximity']]



,Launch Site,Lat,Long,distance_to_proximity
0,CCAFS LC-40,28.562302,-80.577356,6.984382
1,CCAFS LC-40,28.562302,-80.577356,6.984382
2,CCAFS LC-40,28.562302,-80.577356,6.984382
3,CCAFS LC-40,28.562302,-80.577356,6.984382
4,CCAFS LC-40,28.562302,-80.577356,6.984382
5,CCAFS LC-40,28.562302,-80.577356,6.984382
6,CCAFS LC-40,28.562302,-80.577356,6.984382
7,CCAFS LC-40,28.562302,-80.577356,6.984382
8,CCAFS LC-40,28.562302,-80.577356,6.984382
9,CCAFS LC-40,28.562302,-80.577356,6.984382


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
import folium
from folium.plugins import MousePosition

# Create the site_map
site_map = folium.Map(location=[28.5721, -80.648], zoom_start=5)

# Formatter for the latitude and longitude
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add MousePosition to get the coordinate for a mouse over on the map
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition plugin to the map
site_map.add_child(mouse_position)

# Display the map
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
import folium
from folium.plugins import MousePosition

# Create a map centered around the launch site (example: Kennedy Space Center)
site_map = folium.Map(location=[28.5721, -80.648], zoom_start=12)

# Formatter for the latitude and longitude
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add MousePosition to get the coordinates for a mouse over on the map
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add the MousePosition plugin to the map
site_map.add_child(mouse_position)

# Display the map
site_map


In [24]:
# Define coordinates for the launch site and the coastline point
launch_site_lat = 28.5721
launch_site_lon = -80.648

coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance using the calculate_distance function
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"The distance between the launch site and the closest coastline is {distance_coastline:.2f} km.")


The distance between the launch site and the closest coastline is 7.52 km.


In [25]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.plugins import MousePosition
from folium import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of Earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Launch site coordinates (Kennedy Space Center)
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Closest coastline coordinates (example)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a map centered around the launch site (Kennedy Space Center)
site_map = folium.Map(location=[28.5721, -80.648], zoom_start=12)

# Formatter for MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add MousePosition plugin
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Create and add a Marker on the closest coastline point with a distance label
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)
import folium
from folium.plugins import MousePosition
from folium import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of Earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Launch site coordinates (Kennedy Space Center)
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Closest coastline coordinates (example)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a map centered around the launch site (Kennedy Space Center)
site_map = folium.Map(location=[28.5721, -80.648], zoom_start=12)

# Formatter for MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add MousePosition plugin
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Create and add a Marker on the closest coastline point with a distance label
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Create a Polyline between the launch site and the coastline point
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site
    [coastline_lat, coastline_lon]  # Coastline point
]

# Create the PolyLine object
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add Polyline to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [27]:
import folium
from folium.plugins import MousePosition
from folium import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of Earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates of the launch site (Kennedy Space Center)
launch_site_lat = 28.5721
launch_site_lon = -80.648

# Example: Coordinates of a nearby city (e.g., Cocoa Beach)
city_lat = 28.3922
city_lon = -80.6073

# Calculate the distance between the launch site and the city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a map centered around the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Formatter for MousePosition
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add MousePosition plugin
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Create and add a Marker on the city with a distance label
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#3498db;"><b>{:.2f} KM</b></div>'.format(distance_city),
    )
)

# Add the city marker to the map
city_marker.add_to(site_map)

# Create a Polyline between the launch site and the city point
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site
    [city_lat, city_lon]  # City coordinates
]

# Create the PolyLine object
line_to_city = folium.PolyLine(locations=coordinates, weight=2, color='green')

# Add Polyline to the map
site_map.add_child(line_to_city)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


## 1. Are launch sites in close proximity to railways?
Method: First, determine the coordinates of nearby railways (using a dataset or manually identifying known locations of railways near launch sites). Then, calculate and visualize the distances between the launch sites and these railway coordinates.
Interpretation: If the distance is relatively short (for example, within a few kilometers), then launch sites are in close proximity to railways. If the distance is large (e.g., over 50 km), the launch sites may not be in close proximity.
## 2. Are launch sites in close proximity to highways?
Method: Identify the nearest highways (either through geospatial datasets or known highway locations). Use the same distance calculation method as for the railways.
Interpretation: Highways are typically major infrastructure routes, so if the launch sites are close to highways (less than 10 km), it may indicate easy accessibility for logistical purposes. Larger distances suggest that launch sites are situated far from highways.
## 3. Are launch sites in close proximity to coastline?
Method: The coastline proximity is a significant factor for some spaceports (e.g., to reduce risks to populated areas). We can use known coastline coordinates (like for Kennedy Space Center) or use coastlines datasets.
Interpretation: If the launch site is near the coast (less than 10 km), this proximity is often strategic to reduce the risk to the public in case of a rocket malfunction during launch. If the distance is large, it might indicate that the launch site is further inland for some other reason (e.g., to avoid certain environmental or logistical concerns).
## 4. Do launch sites keep certain distance away from cities?
Method: Given the risks involved in space launches, most launch sites are positioned far away from major cities. We can analyze the distances between launch sites and the nearest cities.
Interpretation: If the distance between a launch site and the nearest city is large (e.g., over 20-30 km), this would indicate that spaceports are purposefully positioned far from cities to minimize the potential impact of accidents or malfunctions.
## Steps for Analysis and Findings:
####Proximity to Railways: You would look at the distance calculated between each launch site and the closest railway. If the distance is small, it could suggest easy access for transporting materials. For example, Kennedy Space Center in Florida is not very close to major railways, but other locations might be strategically near railways for logistical reasons.

#### Proximity to Highways:
Similar to railways, you would measure the distance to the closest highway. Launch sites like Cape Canaveral are known for their relatively accessible location, often near major highways. If distances to highways are large, it suggests that the launch site is somewhat isolated, likely for safety reasons.

#### Proximity to Coastline:
Launch sites like Kennedy Space Center are located right on the coast, specifically to mitigate the risk to populated areas during launches. If the launch site is along the coastline (within a few kilometers), this proximity is a deliberate decision based on safety and logistics. On the other hand, if the launch site is inland, it might be due to different environmental factors, or to ensure safe evacuation routes for people in case of emergencies.

#### Distance from Cities:
Launch sites are often situated far from cities to reduce risks. Kennedy Space Center, for example, is situated far from major population centers. Other launch sites like Vandenberg Space Force Base are also in isolated areas. The larger the distance to cities, the better it is in terms of safety concerns, and this is usually a key consideration when building a launch facility.



# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
